In [1]:
import pdfplumber
import pandas as pd
import re
import os

In [2]:
# Store the .pdf file dir into a var
dir = '../data'

In [3]:
# Iterate under dir data to have all files 
for filename in os.listdir(dir):
    # Condition to confirm which file has .pdf extension
    if filename.endswith('.pdf'):
        # Join the dir + filename when .pdf extension
        pdf_file_fullpath = os.path.join(dir, filename)
        # Print the result
        print(pdf_file_fullpath)

../data\part-number-list-2025-08-08.pdf


In [4]:
# Read the pdf document
with pdfplumber.open(pdf_file_fullpath) as pdf:
    # Get the page number
    page = pdf.pages[0]
    # Extract the table (s) 
    table = page.extract_table()

In [5]:
# Get the type
type(page)

pdfplumber.page.Page

In [6]:
# Show the data type of the method pages to find out an method iterable to print all pages
# The data type is a list with all pages, which is possible to interate over this method to extract the table
type(pdf.pages)

list

In [7]:
# Check the data type of the data extracted
type(table)

list

In [8]:
# The data extracted is saved into a list of lists
# Print the first item of the list
# Position 0 at the table are the column names
print(table[0])

['CALIBRATION', 'TYPE', 'OLD PART NUMBER(S)', 'NEW PART NUMBER(s)', 'TSB(S)', 'RECALL(S)']


In [9]:
# List to store all data from all pages, otherwise when iterating over each line (WITHOUT THE LIST), 
# the previous page is removed and the new page is stored in the variable 'table_extracted', 
# that's why only the last 2 lines appears in the final resultWITHOUT LIST  
all_tables = []

# Loop to iterate over the pdf.pages list
for page in pdf.pages:
    # Extract the data from each page
    table_extracted = page.extract_table()
    all_tables.extend(table_extracted)

In [10]:
# Convert the list to dataframe
df = pd.DataFrame(all_tables[1:], columns=all_tables[0])

In [11]:
# Check what rows the str 'CALIBRATION' is located
ser_calibration = df['CALIBRATION'].str.contains('CALIBRATION')

In [12]:
# Get the indexes with the rows containing the str 'CALIBRATION'
index_true = ser_calibration[ser_calibration == True].index

In [13]:
# Drop all indexes with the header 'CALIBRATION', 'TYPE', 'OLD PART NUMBER(S)', 'NEW PART NUMBER(s)', 'TSB(S)', 'RECALL(S)
df_calibration_deleted = df.drop(index=index_true)

In [14]:
# Drop the 2 last columns
df_dropped_columns = df_calibration_deleted.drop(columns=['TSB(S)', 'RECALL(S)'])

In [ ]:
# Regex pattern for BODY CODE:
# ^ - at the beginning of the string
# \n - literal newline
# [A-Z0-9] - upper letter and number regardless the order
# {2} - capture exactly 2 letters
# \s+ - \s whitespace + means more than one whitespace
# - - Hifen literal
body_code_pattern = r'^(?:\d{2,4}\s+)+(?:[\d.]+L?\s+)?([A-Z][A-Z0-9])'

In [ ]:
# # Regex pattern for ENGINE SIZE ('2.5L' or '3.6'):
# [0-9] - first digit
# \.[0-9] - period (.) literal + any digit
# L? - WITH or WITHOUT L
engine_size_pattern = r'\d\.\d+L'

In [15]:
def extract_year(df, pattern, replace_dict=None):
    '''Function to capture the data using the stabilished 
    pattern for year extraction'''

    # Extract years returning a series
    ser_year_extracted = df['CALIBRATION'].str.extract(pattern)[0]

    # Update the main df with the data extracted only where the row is na
    df.loc[df['Year'].isna(), 'Year'] = ser_year_extracted

    # Since replace method will not be used everytime,
    # replace only when there are values for replacing
    if replace_dict is not None:
        df['Year'] = df['Year'].replace(replace_dict)

    # Return the df
    return df

### Year: 95

In [16]:
# Adding column year to the df
df_dropped_columns['Year'] = None

In [17]:
pattern_95 = r'(^95(?=\s\d.))'

In [18]:
# Dict to the correct year format
dict_95 = {
    "95": "1995"
}

In [19]:
df_extracted_95 = extract_year(df_dropped_columns, pattern_95, dict_95)

### Year: 96

In [20]:
pattern_96 = r'(^96(?=\s[A-Z0-9]))'

In [21]:
# Dict to the correct year format
dict_96 = {
    "96": "1996"
}

In [22]:
df_extracted_96 = extract_year(df_extracted_95, pattern_96, dict_96)

### Year: 97

In [23]:
pattern_97 = r'(^97(?=\s[A-Z0-9(]))'

In [24]:
# Dict to the correct year format
dict_97 = {
    "97": "1997"
}

In [25]:
df_extracted_97 = extract_year(df_extracted_96, pattern_97, dict_97)

### Year: 1997

In [26]:
pattern_1997 = r'(^1997(?=\s[A-Z]))'

In [27]:
df_extracted_1997 = extract_year(df_extracted_97, pattern_1997)

### Year: 98

In [28]:
pattern_98 = r'(^98(?=\s[A-Z0-9]))'

In [29]:
# Dict to the correct year format
dict_98 = {
    "98": "1998"
}

In [30]:
df_extracted_98 = extract_year(df_extracted_1997, pattern_98, dict_98)

### Year: 98/99

In [31]:
pattern_98_99 = r'(^98/99(?=\s[A-Z0-9]))'

In [32]:
# Dict to the correct year format
dict_98_99 = {
    "98/99": "1998, 1999"
}

In [33]:
df_extracted_98_99 = extract_year(df_extracted_98, pattern_98_99, dict_98_99)

### Year: 99

In [34]:
pattern_99 = r'(^99(?=\s[A-Z0-9]))'

In [35]:
# Dict to the correct year format
dict_99 = {
    "99": "1999"
}

In [36]:
df_extracted_99 = extract_year(df_extracted_98_99, pattern_99, dict_99)

### Year: 99-2000

In [37]:
pattern_99_2000 = r'(^99-2000(?=\s[A-Z]))'

In [38]:
# Dict to the correct year format
dict_99_2000 = {
    "99-2000": "1999, 2000"
}

In [39]:
df_extracted_99_2000 = extract_year(df_extracted_99, pattern_99_2000, dict_99_2000)

### Year: 2000, 01

In [40]:
pattern_2000_01 = r'(^2000, 01(?=\s[A-Z]))'

In [41]:
# Dict to the correct year format
dict_2000_01 = {
    "2000, 01": "2000, 2001"
}

In [42]:
df_extracted_2000_01 = extract_year(df_extracted_99_2000, pattern_2000_01, dict_2000_01)

### Year: 2000

In [43]:
pattern_2000 = r'(^2000(?=\s[A-Z0-9]))'

In [44]:
df_extracted_2000 = extract_year(df_extracted_2000_01, pattern_2000)

### Year: 2001

In [45]:
pattern_2001 = r'(^2001(?=\s[A-Z0-9]))'

In [46]:
df_extracted_2001 = extract_year(df_extracted_2000, pattern_2001)

### Year: 2001 (2000.5) ## NOT CAPTURED

In [47]:
pattern_2001_2000_5 = r'(^2001 (2000.5)(?=\s[A-Z]))'

In [48]:
dict_2001_2000_5 = {
    "2001 (2000.5)": "2000, 2001"
}

In [49]:
df_extracted_2001_2000_5 = extract_year(df_extracted_2001, pattern_2001_2000_5)

### Year: 2002

In [50]:
pattern_2002 = r'(^2002(?=\s[A-Z0-9]))'

In [51]:
df_extracted_2002 = extract_year(df_extracted_2001_2000_5, pattern_2002)

### Year: 2002 - 2004

In [52]:
pattern_2002_2004 = r'(^2002 - 2004(?=\s[A-Z]))'

In [53]:
dict_2002_2004 = {
    "2002 - 2004": "2002, 2003, 2004"
}

In [54]:
df_extracted_2002_2004 = extract_year(df_extracted_2002, pattern_2002_2004, dict_2002_2004)

### Year: 2003 2004

In [55]:
pattern_2003_2004 = r'(^2003 2004(?=\s[A-Z]))'

In [56]:
dict_2003_2004 = {
    "2003 2004": "2003, 2004"
}

In [57]:
df_extracted_2003_2004 = extract_year(df_extracted_2002_2004, pattern_2003_2004, dict_2003_2004)

### Year: 2003

In [58]:
pattern_2003 = r'(^2003(?=\s[A-Z0-9]))'

In [59]:
df_extracted_2003 = extract_year(df_extracted_2003_2004, pattern_2003)

### Year: 2003-2004

In [60]:
pattern_2003_2004 = r'(^2003-2004(?=\s[A-Z0-9]))'

In [61]:
dict_2003_2004 = {
    '2003-2004': '2003, 2004'    
}

In [62]:
df_extracted_2003_2004 = extract_year(df_extracted_2003, pattern_2003_2004, dict_2003_2004)

### Year: 2003.5

In [63]:
pattern_2003_5 = r'(^2003.5(?=\s[A-Z0-9]))'

In [64]:
df_extracted_2003_5 = extract_year(df_extracted_2003_2004, pattern_2003_5)

### Year: 2004

In [65]:
pattern_2004 = r'(^2004(?=\s[A-Z]))'

In [66]:
df_extracted_2004 = extract_year(df_extracted_2003_5, pattern_2004)

### Year: 2004 - 2006

In [67]:
pattern_2004_2006 = r'(^2004 - 2006(?=\s[A-Z]))'

In [68]:
dict_2004_2006 = {
    "2004 - 2006": "2004, 2005, 2006"    
}

In [69]:
df_extracted_2004_2006 = extract_year(df_extracted_2004, pattern_2004_2006, dict_2004_2006)

### Year: 2004.5

In [70]:
pattern_2004_5 = r'(^2004.5(?=\s[A-Z0-9]))'

In [71]:
df_extracted_2004_5 = extract_year(df_extracted_2004_2006, pattern_2004_5)

### Year: 2005

In [72]:
pattern_2005 = r'(^2005(?=\s[A-Z]))'

In [73]:
df_extracted_2005 = extract_year(df_extracted_2004_5, pattern_2005)

### Year: 2005 2006

In [74]:
pattern_2005_ws_2006 = r'(^2005 2006(?=\s[A-Z]))'

In [75]:
dict_2005_ws_2006 = {
    "2005 2006": "2005, 2006"    
}

In [76]:
df_extracted_2005_h_2006 = extract_year(df_extracted_2005, pattern_2005_ws_2006, dict_2005_ws_2006)

### Year: 2005 - 2007

In [77]:
pattern_2005_2007 = r'(^2005 - 2007(?=\s[A-Z]))'

In [78]:
dict_2005_2007 = {
    "2005 - 2007": "2005, 2006, 2007"    
}

In [79]:
df_extracted_2005_2007 = extract_year(df_extracted_2005_h_2006, pattern_2005_2007, dict_2005_2007)

### Year: 2005 - 2006

In [80]:
pattern_2005_h_2006 = r'(^2005 - 2006(?=\s[A-Z]))'

In [81]:
dict_2005_h_2006 = {
    "2005 - 2006": "2005, 2006"    
}

In [82]:
df_extracted_2005_h_2006 = extract_year(df_extracted_2005_2007, pattern_2005_h_2006, dict_2005_h_2006)

### Year: 2005.5

In [83]:
pattern_2005_5 = r'(^2005.5(?=\s[A-Z]))'

In [84]:
df_extracted_2005_5 = extract_year(df_extracted_2005_h_2006, pattern_2005_5)

### Year: 2006

In [85]:
pattern_2006 = r'(^2006(?=\s[A-Z]))'

In [86]:
df_extracted_2006 = extract_year(df_extracted_2005_5, pattern_2006)

### Year: 2006 2007

In [87]:
pattern_2006_2007 = r'(^2006 2007(?=\s[A-Z]))'

In [88]:
dict_2006_2007 = {
    "2006 2007": "2006, 2007"
}

In [89]:
df_extracted_2006_2007 = extract_year(df_extracted_2006, pattern_2006_2007, dict_2006_2007)

### Year: 2007 2008

In [90]:
pattern_2007_2008 = r'(^2007 2008(?=\s[A-Z]))'

In [91]:
dict_2007_2008 = {
    "2007 2008": "2007, 2008"
}

In [92]:
df_extracted_2007_2008 = extract_year(df_extracted_2006_2007, pattern_2007_2008, dict_2007_2008)

### Year: 2007 2008 2009

In [93]:
pattern_2007_2008_2009 = r'^(2007 2008 2009(?=\s[A-Z]))'

In [94]:
dict_2007_2008_2009 = {
    "2007 2008 2009": "2007, 2008, 2009"
}

In [95]:
df_extracted_2007_2008_2009 = extract_year(df_extracted_2007_2008, pattern_2007_2008_2009, dict_2007_2008_2009)

### Year: 2007 - 2008

In [96]:
pattern_2007_wshws_2008 = r'(^2007 - 2008(?=\s[A-Z]))'

In [97]:
dict_2007_wshws_2008 = {
    "2007 - 2008": "2007, 2008"    
}

In [98]:
df_extracted_2007_wshws_2008 = extract_year(df_extracted_2007_2008_2009, pattern_2007_wshws_2008, dict_2007_wshws_2008)

### Year: 2007.5

In [99]:
pattern_2007_5 = r'(^2007.5(?=\s[A-Z]))'

In [100]:
df_extracted_2007_5 = extract_year(df_extracted_2007_wshws_2008, pattern_2007_5)

### Year: 2007-2009

In [101]:
pattern_2007_h_2009 = r'(^2007-2009(?=\s[A-Z]))'

In [102]:
dict_2007_h_2009 = {
    "2007-2009": "2007, 2008, 2009"    
}

In [103]:
df_extracted_2007_h_2009 = extract_year(df_extracted_2007_5, pattern_2007_h_2009, dict_2007_h_2009)

### Year: 2007

In [104]:
pattern_2007 = r'(^2007(?=\s[A-Z]))'

In [105]:
df_extracted_2007 = extract_year(df_extracted_2007_h_2009, pattern_2007)

In [106]:
# For an unknown reason, the regex is capturing the whitespace after 2007.
# Remove whitespaces at the end of the string
df_extracted_2007['Year'] = df_extracted_2007['Year'].str.strip()

### Year: 2008 2009

In [107]:
pattern_2008_ws_2009 = r'(^2008 2009(?=\s[A-Z]))'

In [108]:
dict_2008_ws_2009 = {
    "2008 2009": "2008, 2009"
}

In [109]:
df_extracted_2008_ws_2009 = extract_year(df_extracted_2007, pattern_2008_ws_2009, dict_2008_ws_2009)

### Year: 2008.5

In [110]:
pattern_2008_5 = r'(^2008.5(?=\s[A-Z]))'

In [111]:
df_extracted_2008_5 = extract_year(df_extracted_2008_ws_2009, pattern_2008_5)

### Year: 2008-2009

In [112]:
pattern_2008_h_2009 = r'(^2008-2009(?=\s[A-Z0-9]))'

In [113]:
dict_2008_h_2009 = {
    "2008-2009": "2008, 2009"    
}

In [114]:
df_extracted_2008_h_2009 = extract_year(df_extracted_2008_5, pattern_2008_h_2009, dict_2008_h_2009)

### Year: 2008-2010

In [115]:
pattern_2008_h_2010 = r'(^2008-2010(?=\s[A-Z]))'

In [116]:
dict_2008_h_2010 = {
    "2008-2010": "2008, 2009, 2010"
}

In [117]:
df_extracted_2008_h_2010 = extract_year(df_extracted_2008_h_2009, pattern_2008_h_2010, dict_2008_h_2010)

### Year: 2008

In [118]:
pattern_2008 = r'(^2008(?=\s[A-Z0-9]))'

In [119]:
df_extracted_2008 = extract_year(df_extracted_2008_h_2010, pattern_2008)

### Year: 2009

In [120]:
pattern_2009 = r'(^2009(?=\s[A-Z0-9]))'

In [121]:
df_extracted_2009 = extract_year(df_extracted_2008, pattern_2009)

### Year: 2009 2010

In [122]:
pattern_2009_ws_2010 = r'(^2009 2010(?=\s[A-Z]))'

In [123]:
dict_2009_ws_2010 = {
    "2009 2010": "2009, 2010"    
}

In [124]:
df_extracted_2009_ws_2010 = extract_year(df_extracted_2009, pattern_2009_ws_2010)

### Year: 2010

In [125]:
pattern_2010 = r'(^2010(?=\s[A-Z0-9]))'

In [126]:
df_extracted_2010 = extract_year(df_extracted_2009_ws_2010, pattern_2010)

### Year: 2011 - 2017

In [127]:
pattern_2011_h_2017 = r'(^2011 - 2017(?=\s[A-Z0-9]))' 

In [128]:
dict_2011_h_2017 = {
    "2011 - 2017": "2011, 2012, 2013, 2014, 2015, 2016, 2017"    
}

In [129]:
df_extracted_2011_h_2017 = extract_year(df_extracted_2010, pattern_2011_h_2017, dict_2011_h_2017)

### Year: 2011.5

In [130]:
pattern_2011_5 = r'(^2011.5(?=\s[A-Z0-9]))'

In [131]:
df_extracted_2011_5 = extract_year(df_extracted_2011_h_2017, pattern_2011_5)

### Year: 2011-2013

In [132]:
pattern_2011_h_2013 = r'(^2011-2013(?=\s[A-Z0-9]))'

In [133]:
dict_2011_h_2013 = {
    "2011-2013": "2011, 2012, 2013"
}

In [134]:
df_extracted_2011_h_2013 = extract_year(df_extracted_2011_5, pattern_2011_h_2013, dict_2011_h_2013)

### Year: 2011 - 2013

In [135]:
pattern_2011_wshws_2013 = r'(^2011 - 2013(?=\s[A-Z0-9]))'

In [136]:
dict_2011_wshws_2013 = {
    "2011 - 2013": "2011, 2012, 2013"
}

In [137]:
df_extracted_2011_wshws_2013 = extract_year(df_extracted_2011_h_2013, pattern_2011_wshws_2013, dict_2011_wshws_2013)

### Yeae: 2011 2012

In [138]:
pattern_2011_2012 = r'(^2011 2012(?=\s[A-Z]))'

In [139]:
dict_2011_2012 = {
    "2011 2012": "2011, 2012"
} 

In [140]:
df_extracted_2011_2012 = extract_year(df_extracted_2011_wshws_2013, pattern_2011_2012, dict_2011_2012)

### Year: 2011 2012 2013

In [141]:
pattern_2011_2012_2013 = r'(^2011 2012 2013(?=\s[A-Z]))'

In [142]:
dict_2011_2012_2013 = {
    "2011 2012 2013": "2011, 2012, 2013"
}

In [143]:
df_extracted_2011_2012_2013 = extract_year(df_extracted_2011_2012, pattern_2011_2012_2013, dict_2011_2012_2013)

### Year: 2011 - 2014

In [144]:
pattern_2011_h_2014 = r'(^2011 - 2014(?=\s[A-Z]))'

In [145]:
dict_2011_h_2014 = {
    "2011 - 2014": "2011, 2012, 2013, 2014"    
}

In [146]:
df_extracted_2011_h_2014 = extract_year(df_extracted_2011_2012_2013, pattern_2011_h_2014, dict_2011_h_2014)

### Year: 2011

In [147]:
pattern_2011 = r'(^2011(?=\s[A-Z0-9]))'

In [148]:
df_extracted_2011 = extract_year(df_extracted_2011_h_2014, pattern_2011)

### Year: 2010 THRU 2012 ## NOT CAPTURED

In [149]:
pattern_2010_thru_2012 = r'(^2010 THRU 2012(?=\s[A-Z]))'

In [150]:
dict_2010_thru_2012 = {
    "2010 THRU 2012": "2010, 2011, 2012"
}

In [151]:
df_extracted_2010_thru_2012 = extract_year(df_extracted_2011, pattern_2010_thru_2012)

### Year: 2012 2013

In [152]:
pattern_2012_2013 = r'(^2012 2013(?=\s[A-Z]))'

In [153]:
dict_2012_2013 = {
    "2012 2013": "2012, 2013"
}

In [154]:
df_extracted_2012_2013 = extract_year(df_extracted_2010_thru_2012, pattern_2012_2013, dict_2012_2013)

### Year: 2012-2013

In [155]:
pattern_2012_h_2013 = r'(^2012-2013(?=\s[A-Z]))'

In [156]:
dict_2012_h_2013 = {
    "2012-2013": "2012, 2013"
}

In [157]:
df_extracted_2012_h_2013 = extract_year(df_extracted_2012_2013, pattern_2012_h_2013, dict_2012_h_2013)

### Year: 2012 - 2013

In [158]:
pattern_2012_wshws_2013 = r'(^2012 - 2013(?=\s[A-Z]))'

In [159]:
dict_2012_wshws_2013 = {
    "2012 - 2013": "2012, 2013"    
}

In [160]:
df_extracted_2012_wshws_2013 = extract_year(df_extracted_2012_h_2013, pattern_2012_wshws_2013, dict_2012_wshws_2013)

### Year: 2012 - 2015

In [161]:
pattern_2012_wshws_2015 = r'(^2012 - 2015(?=\s[A-Z]))'

In [162]:
dict_2012_wshws_2015 = {
    '2012 - 2015': '2012, 2013, 2014, 2015'    
}

In [163]:
df_extracted_2012_wshws_2015 = extract_year(df_extracted_2012_wshws_2013, pattern_2012_wshws_2015, dict_2012_wshws_2015)

### Year: 2012-2015

In [164]:
pattern_2012_h_2015 = r'(^2012-2015(?=\s[A-Z]))'

In [165]:
dict_2012_h_2015 = {
    '2012-2015': '2012, 2013, 2014, 2015'    
}

In [166]:
df_extracted_2012_h_2015 = extract_year(df_extracted_2012_wshws_2015, pattern_2012_h_2015, dict_2012_h_2015)

### Year: 2012 - 13

In [167]:
pattern_2012_13 = r'(^2012 - 13(?=\s[A-Z]))'

In [168]:
dict_2012_13 = {
    "2012 - 13": "2012, 2013"    
}

In [169]:
df_extracted_2012_13 = extract_year(df_extracted_2012_h_2015, pattern_2012_13, dict_2012_13)

### Year: 2012 - 2017

In [170]:
pattern_2012_h_2017 = r'(^2012 - 2017(?=\s[A-Z]))'

In [171]:
dict_2012_h_2017 = {
    "2012 - 2017": "2012, 2013, 2014, 2015, 2016, 2017"    
}

In [172]:
df_extracted_2012_h_2017 = extract_year(df_extracted_2012_13, pattern_2012_h_2017, dict_2012_h_2017)

### Year: 2012

In [173]:
pattern_2012 = r'(^2012(?=\s[A-Z]))'

In [174]:
df_extracted_2012 = extract_year(df_extracted_2012_h_2017, pattern_2012)

### Year: 2013

In [175]:
pattern_2013 = r'(^2013(?=\s[A-Z0-9]))'

In [176]:
df_extracted_2013 = extract_year(df_extracted_2012, pattern_2013)

### Year: 2013 - 2019

In [177]:
pattern_2013_h_2019 = r'(^2013 - 2019(?=\s[A-Z]))'

In [178]:
dict_2013_h_2019 = {
    "2013 - 2019": "2013, 2014, 2015, 2016, 2017, 2018, 2019"    
}

In [179]:
df_extracted_2013_h_2019 = extract_year(df_extracted_2013, pattern_2013_h_2019, dict_2013_h_2019)

### Year: 2013 2014

In [180]:
pattern_2013_ws_2014 = r'(^2013 2014(?=\s[A-Z]))'

In [181]:
dict_2013_ws_2014 = {
    "2013 2014": "2013, 2014"
}

In [182]:
df_extracted_2013_ws_2014 = extract_year(df_extracted_2013_h_2019, pattern_2013_ws_2014, dict_2013_ws_2014)

### Year: 2013 - 2015

In [183]:
pattern_2013_h_2015 = r'(^2013 - 2015(?=\s[A-Z]))'

In [184]:
dict_2013_h_2015 = {
    "2013 - 2015": "2013, 2014, 2015"    
}

In [185]:
df_extracted_2013_h_2015 = extract_year(df_extracted_2013_ws_2014, pattern_2013_h_2015, dict_2013_h_2015)

### Year: 2013 - 2017

In [186]:
pattern_2013_h_2017 = r'(^2013 - 2017(?=\s[A-Z]))'

In [187]:
dict_2013_h_2017 = {
    "2013 - 2017": "2013, 2014, 2015, 2016, 2017"  
}

In [188]:
df_extracted_2013_h_2017 = extract_year(df_extracted_2013_h_2015, pattern_2013_h_2017, dict_2013_h_2017)

### Year: 2014 2015 2016

In [189]:
pattern_2014_2015_2016 = r'(^2014 2015 2016(?=\s[A-Z]))'

In [190]:
dict_2014_2015_2016 = {
    "2014 2015 2016": "2014, 2015, 2016"
}

In [191]:
df_extracted_2014_2015_2016 = extract_year(df_extracted_2013_h_2017, pattern_2014_2015_2016, dict_2014_2015_2016)

### Year: 2014 - 2016

In [192]:
pattern_2014_wshws_2016 = r'(^2014 - 2016(?=\s[A-Z]))'

In [193]:
dict_2014_wshws_2016 = {
    "2014 - 2016": "2014, 2015, 2016"
}

In [194]:
df_extracted_2014_wshws_2016 = extract_year(df_extracted_2014_2015_2016, pattern_2014_wshws_2016, dict_2014_wshws_2016)

### Year: 2014 2015

In [195]:
pattern_2014_2015 = r'(^2014 2015(?=\s[A-Z]))'

In [196]:
dict_2014_2015 = {
    "2014 2015": "2014, 2015"
}

In [197]:
df_extracted_2014_2015 = extract_year(df_extracted_2014_wshws_2016, pattern_2014_2015, dict_2014_2015)

### Year: 2014 - 2015

In [198]:
pattern_2014_h_2015 = r'(^2014 - 2015(?=\s[A-Z]))'

In [199]:
dict_2014_h_2015 = {
    "2014 - 2015": "2014, 2015"
}

In [200]:
df_extracted_2014_h_2015 = extract_year(df_extracted_2014_2015, pattern_2014_h_2015, dict_2014_h_2015)

### Year: 2014-2016

In [201]:
pattern_2014_h_2016 = r'(^2014-2016(?=\s[A-Z]))'

In [202]:
dict_2014_h_2016 = {
    "2014-2016": "2014, 2015, 2016"    
}

In [203]:
df_extracted_2014_h_2016 = extract_year(df_extracted_2014_h_2015, pattern_2014_h_2016, dict_2014_h_2016)

### Year: 2014

In [204]:
pattern_2014 = r'(^2014(?=\s[A-Z]))'

In [205]:
df_extracted_2014 = extract_year(df_extracted_2014_h_2016, pattern_2014)

### Year: 2015

In [206]:
pattern_2015 = r'(^2015(?=\s[A-Z]))'

In [207]:
df_extracted_2015 = extract_year(df_extracted_2014, pattern_2015)

### Year: 2015 2016

In [208]:
pattern_2015_2016 = r'(^2015 2016(?=\s[A-Z]))'

In [209]:
dict_2015_2016 = {
    "2015 2016": "2015, 2016"    
}

In [210]:
df_extracted_2015_2016 = extract_year(df_extracted_2015, pattern_2015_2016, dict_2015_2016)

### Year: 2015 - 2018

In [211]:
pattern_2015_2018 = r'(^2015 - 2018(?=\s[A-Z]))'

In [212]:
dict_2015_2018 = {
    "2015 - 2018": "2015, 2016, 2017, 2018"    
}

In [213]:
df_extracted_2015_2018 = extract_year(df_extracted_2015_2016, pattern_2015_2018, dict_2015_2018)

### Year: 2015 - 2016

In [214]:
pattern_2015_h_2016 = r'(^2015 - 2016(?=\s[A-Z]))'

In [215]:
dict_2015_h_2016 = {
    "2015 - 2016": "2015, 2016"    
}

In [216]:
df_extracted_2015_h_2016 = extract_year(df_extracted_2015_2018, pattern_2015_h_2016, dict_2015_h_2016)

### Year: 2015-2017

In [217]:
pattern_2015_h_2017 = r'(^2015-2017(?=\s[A-Z]))'

In [218]:
dict_2015_h_2017 = {
    "2015-2017": "2015, 2016, 2017"
}

In [219]:
df_extracted_2015_h_2017 = extract_year(df_extracted_2015_h_2016, pattern_2015_h_2017, dict_2015_h_2017)

### Year: 2015 - 2020

In [220]:
pattern_2015_h_2020 = r'(^2015 - 2020(?=\s[A-Z]))'

In [221]:
dict_2015_h_2020 = {
    "2015 - 2020": "2015, 2016, 2017, 2018, 2019, 2020"    
}

In [222]:
df_extracted_2015_h_2020 = extract_year(df_extracted_2015_h_2017, pattern_2015_h_2020, dict_2015_h_2020)

### Year: 2015 - 2023

In [223]:
pattern_2015_2023 = r'(^2015 - 2023(?=\s[A-Z]))'

In [224]:
dict_2015_2023 = {
    "2015 - 2023": "2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023"
}

In [225]:
df_extracted_2015_2023 = extract_year(df_extracted_2015_h_2020, pattern_2015_2023, dict_2015_2023)

### Year: 2015 - 2017

In [226]:
pattern_2015_wshws_2017 = r'(^2015 - 2017(?=\s[A-Z]))'

In [227]:
dict_2015_wshws_2017 = {
    "2015 - 2017": "2015, 2016, 2017"    
}

In [228]:
df_extracted_2015_wshws_2017 = extract_year(df_extracted_2015_2023, pattern_2015_wshws_2017, dict_2015_wshws_2017)

### Year: 2016

In [229]:
pattern_2016 = r'(^2016(?=\s[A-Z]))'

In [230]:
df_extracted_2016 = extract_year(df_extracted_2015_wshws_2017, pattern_2016)

### Year: 2016 - 2018

In [231]:
pattern_2016_h_2018 = r'(^2016 - 2018(?=\s[A-Z]))'

In [232]:
dict_2016_h_2018 = {
    "2016 - 2018": "2016, 2017, 2018"
}

In [233]:
df_extracted_2016_h_2018 = extract_year(df_extracted_2016, pattern_2016_h_2018, dict_2016_h_2018)

### Year: 2016 - 2020

In [234]:
pattern_2016_h_2020 = r'(^2016 - 2020(?=\s[A-Z]))'

In [235]:
dict_2016_h_2020 = {
    "2016 - 2020": "2016, 2017, 2018, 2019, 2020"
}

In [236]:
df_extracted_2016_h_2020 = extract_year(df_extracted_2016_h_2018, pattern_2016_h_2020, dict_2016_h_2020)

### Year: 2016 - 2022

In [237]:
pattern_2016_h_2022 = r'(^2016 - 2022(?=\s[A-Z]))'

In [238]:
dict_2016_h_2022 = {
    "2016 - 2022": "2016, 2017, 2018, 2019, 2020, 2021, 2022"
}

In [239]:
df_extracted_2016_h_2022 = extract_year(df_extracted_2016_h_2020, pattern_2016_h_2022, dict_2016_h_2022)

### Year: 2016 - 2017

In [240]:
pattern_2016_h_2017 = r'(^2016 - 2017(?=\s[A-Z]))'

In [241]:
dict_2016_h_2017 = {
    "2016 - 2017": "2016, 2017"     
}

In [242]:
df_extracted_2016_h_2017 = extract_year(df_extracted_2016_h_2022, pattern_2016_h_2017, dict_2016_h_2017)

### Year: 2016 - 2021

In [243]:
pattern_2016_h_2021 = r'(^2016 - 2021(?=\s[A-Z]))'

In [244]:
dict_2016_h_2021 = {
    "2016 - 2021": "2016, 2017, 2018, 2019, 2020, 2021"
}

In [245]:
df_extracted_2016_h_2021 = extract_year(df_extracted_2016_h_2022, pattern_2016_h_2021, dict_2016_h_2021)

### Year: 2016 - 2019

In [246]:
pattern_2016_h_2019 = r'(^2016 - 2019(?=\s[A-Z]))'

In [247]:
dict_2016_h_2019 = {
    "2016 - 2019": "2016, 2017, 2018, 2019"    
}

In [248]:
df_extracted_2016_h_2019 = extract_year(df_extracted_2016_h_2021, pattern_2016_h_2019, dict_2016_h_2019)

### Year: 2017

In [249]:
pattern_2017 = r'(^2017(?=\s[A-Z]))'

In [250]:
df_extracted_2017 = extract_year(df_extracted_2016_h_2019, pattern_2017)

### Year: 2017 - 2018

In [251]:
pattern_2017_h_2018 = r'(^2017 - 2018(?=\s[A-Z]))'

In [252]:
dict_2017_h_2018 = {
    "2017 - 2018": "2017, 2018"
}

In [253]:
df_extracted_2017_h_2018 = extract_year(df_extracted_2017, pattern_2017_h_2018)

### Year: 2017 - 2019

In [254]:
pattern_2017_h_2019 = r'(^2017 - 2019(?=\s[A-Z]))'

In [255]:
dict_2017_h_2019 = {
    "2017 - 2019": "2017, 2018, 2019"    
}

In [256]:
df_extracted_2017_h_2019 = extract_year(df_extracted_2017_h_2018, pattern_2017_h_2019, dict_2017_h_2019)

### Year: 2017 - 2020

In [257]:
pattern_2017_h_2020 = r'(^2017 - 2020(?=\s[A-Z]))'

In [258]:
dict_2017_h_2020 = {
    "2017 - 2020": "2017, 2018, 2019, 2020"
}

In [259]:
df_extracted_2017_h_2020 = extract_year(df_extracted_2017_h_2019, pattern_2017_h_2020, dict_2017_h_2020)

### Year: 2017 - 2018B1

In [ ]:
pattern_2017_h_2018B1 = r'(^2017 - 2018(?=\s|$))'

In [261]:
dict_2017_h_2018B1 = {
    "2017 - 2018B1": "2017, 2018"
}

In [262]:
df_extracted_2017_h_2018B1 = extract_year(df_extracted_2017_h_2020, pattern_2017_h_2018B1)

### Year: 2017 - 2022

In [ ]:
pattern_2017_h_2022 = r'(^2017 - 2022(?=\s[A-Z])'

In [264]:
dict_2017_h_2022 = {
    "2017 - 2022": "2017, 2018, 2019, 2020, 2021, 2022"
}

In [265]:
df_extracted_2017_h_2022 = extract_year(df_extracted_2017_h_2018B1, pattern_2017_h_2022, dict_2017_h_2022)

### Year: 2017 - 2021

In [ ]:
pattern_2017_h_2021 = r'(^2017 - 2021(?=\s[A-Z]))'

In [267]:
dict_2017_h_2021 = {
    "2017 - 2021": "2017, 2018, 2019, 2020, 2021"
}

In [268]:
df_extracted_2017_h_2021 = extract_year(df_extracted_2017_h_2022, pattern_2017_h_2021, dict_2017_h_2021)

### Year: 2018

In [269]:
pattern_2018 = r'(^2018(?=\s[A-Z]))'

In [270]:
df_extracted_2018 = extract_year(df_extracted_2017_h_2021, pattern_2018)

### Year: 2018 - 2023

In [271]:
pattern_2018_h_2023 = r'(^2018 - 2023(?=\s[A-Z]))'

In [272]:
dict_2018_h_2023 = {
    "2018 - 2023": "2018, 2019, 2020, 2021, 2022, 2023"
}

In [273]:
df_extracted_2018_h_2023 = extract_year(df_extracted_2018, pattern_2018_h_2023)

### Year: 2018 - 2019

In [274]:
pattern_2018_h_2019 = r'(^2018 - 2019(?=\s[A-Z]))'

In [275]:
dict_2018_h_2019 = {
    "2018 - 2019": "2018, 2019"
}

In [276]:
df_extracted_2018_h_2019 = extract_year(df_extracted_2018_h_2023, pattern_2018_h_2019, dict_2018_h_2019)

### Year: 2018 - 2022

In [277]:
pattern_2018_h_2022 = r'(^2018 - 2022(?=\s[A-Z]))'

In [278]:
dict_2018_h_2022 = {
    "2018 - 2022": "2018, 2019, 2020, 2021, 2022"    
}

In [279]:
df_extracted_2018_h_2022 = extract_year(df_extracted_2018_h_2019, pattern_2018_h_2022, dict_2018_h_2022)

### Year: 2018 - 2021

In [280]:
pattern_2018_h_2021 = r'(^2018 - 2021(?=\s[A-Z]))'

In [281]:
dict_2018_h_2021 = {
    "2018 - 2021": "2018, 2019, 2020, 2021"    
}

In [282]:
df_extracted_2018_h_2021 = extract_year(df_extracted_2018_h_2022, pattern_2018_h_2021, dict_2018_h_2021)

### Year: 2018 - 2024

In [283]:
pattern_2018_h_2024 = r'(^2018 - 2024(?=\s[A-Z]))'

In [284]:
dict_2018_h_2024 = {
    "2018 - 2024": "2018, 2019, 2020, 2021, 2022, 2023, 2024"
}

In [285]:
df_extracted_2018_h_2024 = extract_year(df_extracted_2018_h_2021, pattern_2018_h_2024, dict_2018_h_2024)

### Year: 2018 - 2020

In [286]:
pattern_2018_h_2020 = r'(^2018 - 2020(?=\s[A-Z]))'

In [287]:
dict_2018_h_2020 = {
    "2018 - 2020": "2018, 2019, 2020"    
}

In [288]:
df_extracted_2018_h_2020 = extract_year(df_extracted_2018_h_2024, pattern_2018_h_2020)

### Year: 2019

In [289]:
pattern_2019 = r'(^2019(?=\s[A-Z]))'

In [290]:
df_extracted_2019 = extract_year(df_extracted_2018_h_2020, pattern_2019)

### Year: 2019 - 2024

In [291]:
pattern_2019_h_2024 = r'(^2019 - 2024(?=\s[A-Z]))'

In [292]:
dict_2019_h_2024 = {
    "2019 - 2024": "2019, 2020, 2021, 2022, 2023, 2024"    
}

In [293]:
df_extracted_2019_h_2024 = extract_year(df_extracted_2019, pattern_2019_h_2024, dict_2019_h_2024)

### Year: 2019 - 2022

In [294]:
pattern_2019_h_2022 = r'(^2019 - 2022(?=\s[A-Z]))'

In [295]:
dict_2019_h_2022 = {
    "2019 - 2022": "2019, 2020, 2021, 2022"
}

In [296]:
df_extracted_2019_h_2022 = extract_year(df_extracted_2019_h_2024, pattern_2019_h_2022)

### Year: 2019 - 2020

In [297]:
pattern_2019_h_2020 = r'(^2019 - 2020(?=\s[A-Z]))'

In [298]:
dict_2019_h_2020 = {
    "2019 - 2020": "2019, 2020"    
}

In [299]:
df_extracted_2019_h_2020 = extract_year(df_extracted_2019_h_2022, pattern_2019_h_2020, dict_2019_h_2020)

### Year: 2019 - 2021

In [300]:
pattern_2019_h_2021 = r'(^2019 - 2021(?=\s[A-Z]))'

In [301]:
dict_2019_h_2021 = {
    "2019 - 2021": "2019, 2020, 2021"    
}

In [302]:
df_extracted_2019_h_2021 = extract_year(df_extracted_2019_h_2020, pattern_2019_h_2021, dict_2019_h_2021)

### Year: 2019 - 2023

In [303]:
pattern_2019_h_2023 = r'(^2019 - 2023(?=\s[A-Z]))'

In [304]:
dict_2019_h_2023 = {
    "2019 - 2023": "2019, 2020, 2021, 2022, 2023"    
}

In [305]:
df_extracted_2019_h_2023 = extract_year(df_extracted_2019_h_2021, pattern_2019_h_2023, dict_2019_h_2023)

### Year: 2020

In [306]:
pattern_2020 = r'(^2020(?=\s[A-Z]))'

In [307]:
df_extracted_2020 = extract_year(df_extracted_2019_h_2023, pattern_2020)

### Year: 2020 - 2022

In [308]:
pattern_2020_h_2022 = r'(^2020 - 2022(?=\s[A-Z]))'

In [309]:
dict_2020_h_2022 = {
    "2020 - 2022": "2020, 2021, 2022"    
}

In [310]:
df_extracted_2020_h_2022 = extract_year(df_extracted_2020, pattern_2020_h_2022)

### Year: 2020 2022

In [311]:
pattern_2020_2022 = r'(^2020 2022(?=\s[A-Z]))'

In [312]:
dict_2020_2022 = {
    "2020 2022": "2020, 2021, 2022"
}

In [313]:
df_extracted_2020_2022 = extract_year(df_extracted_2020_h_2022, pattern_2020_2022, dict_2020_2022)

### Year: 2020 - 2021

In [314]:
pattern_2020_h_2021 = r'(^2020 - 2021(?=\s[A-Z]))'

In [315]:
dict_2020_h_2021 = {
    "2020 - 2021": "2020, 2021"
}

In [316]:
df_extracted_2020_h_2021 = extract_year(df_extracted_2020_2022, pattern_2020_h_2021)

### Year: 2020 - 2025

In [317]:
pattern_2020_h_2025 = r'(^2020 - 2025(?=\s[A-Z]))'

In [318]:
dict_2020_h_2025 = {
    "2020 - 2025": "2020, 2021, 2022, 2023, 2024, 2025"
}

In [319]:
df_extracted_2020_h_2025 = extract_year(df_extracted_2020_h_2021, pattern_2020_h_2025, dict_2020_h_2025)

### Year: 2020 - 2023

In [320]:
pattern_2020_h_2023 = r'(^2020 - 2023(?=\s[A-Z]))'

In [321]:
dict_2020_h_2023 = {
    "2020 - 2023": "2020, 2021, 2022, 2023"
}

In [322]:
df_extracted_2020_h_2023 = extract_year(df_extracted_2020_h_2025, pattern_2020_h_2023, dict_2020_h_2023)

### Year: 2021

In [323]:
pattern_2021 = r'(^2021(?=\s[A-Z]))' 

In [324]:
df_extracted_2021 = extract_year(df_extracted_2020_h_2023, pattern_2021)

### Year: 2021 - 2022

In [326]:
pattern_2021_h_2022 = r'(^2021 - 2022(?=\s[A-Z]))'

In [327]:
dict_2021_h_2022 = {
    "2021 - 2022": "2021, 2022"
}

In [328]:
df_extracted_2021_h_2022 = extract_year(df_extracted_2021, pattern_2021_h_2022, dict_2021_h_2022)

### Year: 2021 - 2023

In [329]:
pattern_2021_h_2023 = r'(^2021 - 2023(?=\s[A-Z]))'

In [330]:
dict_2021_h_2023 = {
    "2021 - 2023": "2021, 2022, 2023"    
}

In [331]:
df_extracted_2021_h_2023 = extract_year(df_extracted_2021_h_2022, pattern_2021_h_2023, dict_2021_h_2023)

### Year: 2022 - 2023

In [332]:
pattern_2022_h_2023 = r'(^2022 - 2023(?=\s[A-Z]))'

In [333]:
dict_2022_h_2023 = {
    "2022 - 2023": "2022, 2023"    
}

In [334]:
df_extracted_2022_h_2023 = extract_year(df_extracted_2021_h_2023, pattern_2022_h_2023)

### Year: 2022 - 2024

In [335]:
pattern_2022_h_2024 = r'(^2022 - 2024(?=\s[A-Z]))'

In [336]:
dict_2022_h_2024 = {
    "2022 - 2024": "2022, 2023, 2024"    
}

In [337]:
df_extracted_2022_h_2024 = extract_year(df_extracted_2022_h_2023, pattern_2022_h_2024, dict_2022_h_2024)

### Year: 2022 2024

In [338]:
pattern_2022_2024 = r'(^2022 2024(?=\s[A-Z]))'

In [339]:
dict_2022_2024 = {
    "2022 2024": "2022, 2023, 2024"
}

In [340]:
df_extracted_2022_2024 = extract_year(df_extracted_2022_h_2024, pattern_2022_2024, dict_2022_2024)

### Year: 2022 - 2025

In [341]:
pattern_2022_h_2025 = r'(^2022 - 2025(?=\s[A-Z]))'

In [342]:
dict_2022_h_2025 = {
    "2022 - 2025": "2022, 2023, 2024, 2025"
}

In [343]:
df_extracted_2022_2025 = extract_year(df_extracted_2022_2024, pattern_2022_h_2025)

### Year: 2022

In [344]:
pattern_2022 = r'(^2022(?=\s[A-Z]))'

In [345]:
df_extracted_2022 = extract_year(df_extracted_2022_2025, pattern_2022)

### Year: 2023

In [346]:
pattern_2023 = r'(^2023(?=\s[A-Z]))'

In [347]:
df_extracted_2023 = extract_year(df_extracted_2022, pattern_2023)

### Year: 2023 2024

In [348]:
pattern_2023_2024 = r'(^2023 2024(?=\s[A-Z]))'

In [349]:
dict_2023_2024 = {
    "2023 2024": "2023, 2024"
}

In [350]:
df_extracted_2023_2024 = extract_year(df_extracted_2023, pattern_2023_2024, dict_2023_2024)

### Year: 2023 - 2024

In [351]:
pattern_2023_h_2024 = r'(^2023 - 2024(?=\s[A-Z]))'

In [352]:
dict_2023_h_2024 = {
    "2023 - 2024": "2023, 2024"
}

In [353]:
df_extracted_2023_h_2024 = extract_year(df_extracted_2023_2024, pattern_2023_h_2024, dict_2023_h_2024)

### Year: 2023 - 2025

In [354]:
pattern_2023_h_2025 = r'(^2023 - 2025(?=\s[A-Z]))'

In [355]:
dict_2023_h_2025 = {
    "2023 - 2025": "2023, 2024, 2025"    
}

In [356]:
df_extracted_2023_h_2025 = extract_year(df_extracted_2023_h_2024, pattern_2023_h_2025, dict_2023_h_2025)

### Year: 2024

In [357]:
pattern_2024 = r'(^2024(?=\s[A-Z]))'

In [358]:
df_extracted_2024 = extract_year(df_extracted_2023_h_2025, pattern_2024)

### Year: 2024 - 2025

In [359]:
pattern_2024_h_2025 = r'(^2024 - 2025(?=\s[A-Z]))'

In [360]:
dict_2024_h_2025 = {
    "2024 - 2025": "2024, 2025"    
}

In [361]:
df_extracted_2024_h_2025 = extract_year(df_extracted_2024, pattern_2024_h_2025, dict_2024_h_2025)

### Year: 2025

In [362]:
pattern_2025 = r'(^2025(?=\s[A-Z]))'

In [363]:
df_extracted_2025 = extract_year(df_extracted_2024_h_2025, pattern_2025)

### Year: 2025 - 2026

In [364]:
pattern_2025_h_2026 = r'(^2025 - 2026(?=\s[A-Z]))'

In [365]:
dict_2025_h_2026 = {
    "2025 - 2026": "2025, 2026"
}

In [366]:
df_extracted_2025_h_2026 = extract_year(df_extracted_2025, pattern_2025_h_2026, dict_2025_h_2026)

### Year: 2026

In [367]:
pattern_2026 = r'(^2026(?=\s[A-Z]))'

In [368]:
df_extracted_2026 = extract_year(df_extracted_2025_h_2026, pattern_2026)

In [369]:
df_extracted_2026[df_extracted_2026["Year"].isna()]

,CALIBRATION,TYPE,OLD PART NUMBER(S),NEW PART NUMBER(s),Year
585,"99, 00, 01 WJ/WG EATXIV\nWJ - GRAND CHEROKEE",TCM,05019701AA\n05019701AB\n05066376AA\n56041328AF...,05019701AC,NaN
3573,2001 (2000.5) BR/BE 5.9L AUTO FED\nCUMMINSCOLD...,ECM,05083997AA\n313530300J\n333030301J\n333030302J...,313530300J,NaN
3574,2001 (2000.5) BR/BE 5.9L AUTO FED\nCUMMINSCOLD,ECM,,313530300J,NaN
3575,2001 (2000.5) BR/BE 5.9L AUTO FED\nCUMMINSCOLD,ECM,,313530300J,NaN
3623,2001 (2000.5) BR/BE 5.9L AUTO FED\nCUMMINSNORM...,ECM,05083997AA\n30302E36_9\n313530300J\n333030300J...,333030308J,NaN
3624,2001 (2000.5) BR/BE 5.9L AUTO FED\nCUMMINSNORM,ECM,,333030308J,NaN
3625,2001 (2000.5) BR/BE 5.9L AUTO FED\nCUMMINSNORM,ECM,,333030308J,NaN
3626,2001 (2000.5) BR/BE 5.9L HO MAN FED\nCUMMINS\n...,ECM,05083997AA\n333032300J\n333032301J\n333032302J...,333032306J,NaN
3627,2001 (2000.5) BR/BE 5.9L MAN FED CUMMINS\nBE -...,ECM,05083997AA\n333033300J\n333033301J\n333033302J...,333033307J,NaN
3677,2001 (2000.5) BR/BE 5.9L HO MAN CAL\nCUMMINS\n...,ECM,05083997AA\n373930300J\n373930301J\n373930302J...,373930305J,NaN


In [748]:
(df_extracted_2023_h_2024["Year"] == "2023 - 2024").sum()

np.int64(19)

In [ ]:
df_extracted_2008_h_2010.to_csv("C:\Language_Projects\Language_Projects\Python\Flagship_1\pdf_data_extract\data\df_extracted_2008_h_2010.csv", index=True)

In [ ]:
# Print the head to see the results
df_y_extracted.head(5)

In [ ]:
# Include ', ' between year whitespaces
df_year_column = df_y_extracted['Year'].str.replace(' ', ', ', regex=False)

In [ ]:
# Concat column updated
df_y_extracted['Year'] = df_year_column

In [ ]:
# Print the head to see the changes
df_y_extracted.head(5)

In [ ]:
def get_list_of_years(df):
    # List to store the updated years
    processed_years = []

    # Iterate under column year to add 19 to 2 digits years
    for year in df['Year']:
        # Confirm if the year lenght is 2 digits
        if len(year) == 2:
            processed_years.append('19' + year)
        else:
            processed_years.append(year)

    return processed_years

In [ ]:
# Call the function to add '19' to the years with 2 digits
years_updated_list = get_list_of_years(df_y_extracted)

In [ ]:
# Update the df
df_y_extracted['Year'] = years_updated_list

In [ ]:
# Convert the year_list to a unique string, 
# split method converts the unique string to a list of strings separated by ', '
list_of_years = ', '.join(years_updated_list).split(', ')

In [ ]:
# Iterator under the list to confirm what years is different than 4 digits
for year in list_of_years:
    if len(year) != 4:
        print(year)

In [ ]:
# Check which rows have year 2018, 
ser_incorrect_year = df_y_extracted['Year'].str.contains('2018,')

In [ ]:
# Get the indexes
ser_incorrect_year[ser_incorrect_year == True]

In [ ]:
# Print the row with the years to see the problem
df_y_extracted['Year'][3826]

In [ ]:
# Check how it is on the origin row
df_y_extracted['CALIBRATION'][3826]
df_y_extracted['CALIBRATION'][3829]

In [ ]:
# Replace 2 commas by only 1 comma 
df_y_extracted.loc[[3826, 3829], 'Year'] = df_y_extracted.loc[[3826, 3829], 'Year'].str.replace(',, ', ', ', regex=False)

### YEAR CHECKING

In [ ]:
# Get a list with the years
list_of_years_after_correction = get_list_of_years(df_y_extracted)

In [ ]:
# Create a list with years
list_of_updated_years = ', '.join(list_of_years_after_correction).split(', ')

In [ ]:
# Iterator under the list to confirm what years is different than 4 digits
for year in list_of_updated_years:
    if len(year) != 4:
        print(year)

In [ ]:
# Convert the list to df
df_years = pd.DataFrame(list_of_updated_years)

In [ ]:
# Get the indexes that show empty at column 0 (year column)
df_years[df_years[0] == '']

In [ ]:
# Update the empty rows with values to convert all years to int
df_years.loc[[3542, 3547, 3551], 0] = df_years.loc[[3542, 3547, 3551], 0].str.replace('', '0') 

In [ ]:
# List with years converted to int
years_converted_to_int_list = []

# Convert the column 0 to int
for year in df_years[0]:
    # int method to convert all data to int values
    int_year = int(year)
    years_converted_to_int_list.append(int_year)

In [ ]:
# Sort the list
years_converted_to_int_list.sort()

In [ ]:
# Use the method set to get a list with UNIQUE YEARS to check if there are incorrect years.
print(set(years_converted_to_int_list))

### BODY CODE EXTRACTION

In [ ]:
# First body code capture, after line break '\n'
# \n - line break
# [A-Z0-9] - upper letter followed by number
# {2} - 2 digits
# \s+ - 1 or more whitespaces
body_code_pattern_after_line_break = r'\n([A-Z0-9]{2})\s+-'

In [ ]:
def extract_body_code(df, new_column, column, pattern):
    '''Function to capture the data using the stabilished 
    pattern for year, body code and engine size'''
    df[new_column] = df[column].str.findall(pattern, re.MULTILINE)

    # findall method extract the desired data from each row and place it in a new row with list form.
    # .str.join method converts the list to a unique string: 
    # [2001, 2002] -> 2001, 2002 (using the separator ', ') 
    df[new_column] = df[new_column].str.join(', ')

    # Return the df after data extraction
    return df

In [ ]:
# Extract the body code
df_body_code_extracted_pattern_after_line_break = extract_data(df_y_extracted, 'Body', 'CALIBRATION', body_code_pattern_after_line_break)

In [ ]:
# Print the head to see the result
df_body_code_extracted_pattern_after_line_break.head()

In [ ]:
# Body code capture after line break '\n'
# \n - line break
# ([A-Z0-9]{2}) - group of capture to the body code, 2 digits of letters or numbers
# \s-\s - capture exactly whitespace - whitespace
# ([^\n]+) - capture the entire text after - until the next line break
body_code_pattern_descr = r'\n([A-Z0-9]{2}\s-\s[^\n]+)'

In [ ]:
# Call the function to extract the body code and all data after the body code
df_body_code_extracted_pattern_after_line_break['BC Description'] = df_body_code_extracted_pattern_after_line_break['CALIBRATION'].str.findall(body_code_pattern_descr, re.MULTILINE)

In [ ]:
# findall method capture all body code descriptions and return all of them info a list.
# This method extract all from the list to be updated in the same column as strings out of list
# join(', ') method converts the strings INSIDE A LIST IN EACH ROW to a unique string IN EACH ROW  
df_body_code_extracted_pattern_after_line_break['BC Description'] = df_body_code_extracted_pattern_after_line_break['BC Description'].str.join(', ')

In [ ]:
# Filter the empty rows and save in a variable to check for next pattern
df_bc_and_bc_desc = df_body_code_extracted_pattern_after_line_break[df_body_code_extracted_pattern_after_line_break['Body'] == '']

In [ ]:
# Check the size of the df with the body codes captured
len(list(df_body_code_extracted_pattern_after_line_break.index))

In [ ]:
# Counting the remaining rows to capture the body code
len(list(df_bc_and_bc_desc.index))

In [ ]:
# Pattern to capture the body codes at the beginning of the string
# (? - find this group but does not capture
# <= - lookbehind, check if the pattern is before but does not capture
# \d{4}\s) - 4 digits and 1 whitespace
# (?:\s[A-Z0-9]{2})* - capture 0 or more body codes all followed by whitespaces
# \b - make sure finishes at the end of the last body code
body_code_pattern_beginning_string = r'(?<=\d{4}\s)([A-Z0-9]{2}(?:\s[A-Z0-9]{2})*)\b'

In [ ]:
# Call the function to extract the remainig body codes
df_body_code_beginning_string = extract_data(df_bc_and_bc_desc, 'BC', 'CALIBRATION', body_code_pattern_beginning_string)

In [ ]:
# Drop column body
df_body_code_beginning_string_dropped_column = df_body_code_beginning_string.drop(columns=['Body'])

In [ ]:
# Rename the column from BC to Body
df_body_code_beginning_string_renamed_column = df_body_code_beginning_string_dropped_column.rename(columns={'BC': 'Body'})

In [ ]:
# Replace ' ' between thebody codes for ', '
df_body_code_beginning_string_renamed_column['Body'] = df_body_code_beginning_string_renamed_column['Body'].str.replace(' ', ', ', regex=False) 

In [ ]:
# Change the column order to match the main df
df_body_code_beginning_string_renamed_column = df_body_code_beginning_string_renamed_column[['CALIBRATION', 'TYPE', 'OLD PART NUMBER(S)', 'NEW PART NUMBER(s)', 'Year', 'Body', 'BC Description']]

In [ ]:
# Merge the main df with the df with the extracted codes at the beginning of the string
# left_index, right_index - the index will be the key match between both dataframes, such as a match ID on both dfs
# outer: use union of keys from both frames
dfs_body_code_merged = df_body_code_extracted_pattern_after_line_break.merge(df_body_code_beginning_string_renamed_column, how='outer', left_index=True, right_index=True)

In [ ]:
# Check the columns name
dfs_body_code_merged.columns

In [ ]:
# Drop y columns
dfs_body_code_merged_dropped_columns = dfs_body_code_merged.drop(columns=['CALIBRATION_y', 'TYPE_y', 'OLD PART NUMBER(S)_y',
       'NEW PART NUMBER(s)_y', 'Year_y', 'BC Description_y'])

In [ ]:
# Replace empty strings at column Body_x
# ^$ - string vazia, nada antes e nada depis
# str.replace() - used ONLY with strings
dfs_body_code_merged_dropped_columns['Body_x'] = dfs_body_code_merged_dropped_columns['Body_x'].replace(r'^$', pd.NA, regex=True) 

In [ ]:
# Fill the empty body codes at column body_x (main df) from column body_y
dfs_body_code_merged_dropped_columns['Body_x'] = dfs_body_code_merged_dropped_columns['Body_x'].fillna(dfs_body_code_merged_dropped_columns['Body_y'])

In [ ]:
# Remove column body_y 
df_merged_drop_y_column = dfs_body_code_merged_dropped_columns.drop(columns=['Body_y'])

In [ ]:
# Remove _x from columns and return the columns to their original names
df_merged_renamed_x_columns = df_merged_drop_y_column.rename(columns={
    'CALIBRATION_x': 'CALIBRATION',
    'TYPE_x': 'TYPE',
    'OLD PART NUMBER(S)_x': 'OLD PART NUMBER(S)',
    'NEW PART NUMBER(s)_x': 'NEW PART NUMBER(s)',
    'Year_x': 'Year',
    'Body_x': 'Body',
    'BC Description_x': 'BC Description'
    })

In [ ]:
# Confirming how many rows at body column are filled with empty string ''
len(df_merged_renamed_x_columns[df_merged_renamed_x_columns['Body'] == ''])

In [ ]:
# Confirming how many rows are empty
len(df_merged_renamed_x_columns[df_merged_renamed_x_columns['Body'] == pd.NA])

In [ ]:
# Function to create a new column with the body code length
def body_code_len(df):
    # Create a list with the body codes length
    body_code_length = []

    # Loop to get each body code
    for code in df['Body']:
        # Count the lenght of each body code
        bc_len = len(code)
    # Append the lenght
        body_code_length.append(bc_len)

    # Add a body code lenght column to the main df
    df['BC Length'] = body_code_length
    return df 

In [ ]:
# Add a body code lenght column to the main df
df_merged_bc_len = body_code_len(df_merged_renamed_x_columns)

### BODY CODE CHECKING

In [ ]:
# Check the body codes based on their length
df_merged_bc_len['BC Length'].value_counts()

In [ ]:
# Method to capture the body codes individually
bc_pattern = r'[A-Z0-9]{2}'

In [ ]:
# Create a list with unique body codes
unique_bc_list = []

# Iterate under column bc to get the string with the bcs
for code in df_merged_bc_len['Body']:
    # Method findall find the matches based on the pattern
    bc_match = re.findall(bc_pattern, code)
    if code != '':
        unique_bcs = list(set(bc_match))
        bc_out_list = ', '.join(unique_bcs)
        unique_bc_list.append(bc_out_list)
    else:
        unique_bc_list.append(code) 

In [ ]:
# Update the main df
df_merged_bc_len['Body'] = unique_bc_list

In [ ]:
# Call the function to update the bc len
df_bc_len_updated = body_code_len(df_merged_bc_len)

In [ ]:
# Call the function to get body code length
df_bc_len_updated['BC Length'].value_counts()

In [ ]:
# Check the body codes with the description to confirm they are all correct
# Creat a list with all bc_descriptions
bc_descriptions_list = []

# Iterate under the bc descr column to get all bc desc
for desc in df_bc_len_updated['BC Description']:
    bc_descriptions_list.append(desc)

In [ ]:
# Convert the bc description list to a unique string
# split method converts the unique string to a list of string separated by ', '
bc_descr_list = ', '.join(bc_descriptions_list).split(', ')

In [ ]:
# Create a list with unique bc descr
unique_bc_descr_list = set(bc_descr_list)

In [ ]:
# Print the list
print(unique_bc_descr_list)

### ENGINE SIZE EXTRACTION

In [ ]:
# Extract the engine size
df_year_body_engine_size = extract_data(df_bc_len_updated, 'Engine Size', 'CALIBRATION', engine_size_pattern)

In [ ]:
# Print the head to see the results
df_year_body_engine_size.head(5)

In [ ]:
# List with engine sizes updated
engine_size_list = []

# Iterate over column Engine size to add 'L' to engine sizes without L
for engine in df_year_body_engine_size['Engine Size']:
    if len(engine) == 3:
        engine_updated = engine + 'L'
        engine_size_list.append(engine_updated)
    else:
        engine_size_list.append(engine)

In [ ]:
# Update the df with the updated engine sizes
df_year_body_engine_size['Engine Size'] = engine_size_list

In [ ]:
# See the head to see the results
df_year_body_engine_size.head(7)

In [ ]:
# Remove char '\n'
df_year_body_engine_size_removed_char = df_year_body_engine_size.replace(r'\n', ' ', regex=True)

In [ ]:
# Print the head to see the results
df_year_body_engine_size_removed_char.head(7)

In [ ]:
# Add , between whitespaces in the old part number column
df_year_body_engine_size_removed_char['OLD PART NUMBER(S)'] = df_year_body_engine_size_removed_char['OLD PART NUMBER(S)'].str.replace(' ', ', ', regex=False)

In [ ]:
# Print the head to see the results
df_year_body_engine_size_removed_char.head(10)

In [ ]:
# Create a list with all engine sizes to confirm if they are correct.
engine_size_list = []

# Iterate over the engine size column to get the engine size
for es in df_year_body_engine_size_removed_char['Engine Size']:
    engine_size_list.append(es)

In [ ]:
# Convert the engine size list to a unique string,
# split method converts the unique string to a list of strings separated by ', '
list_of_engine_size = ', '.join(engine_size_list).split(', ')

In [ ]:
# Create a list with unique engine size set
unique_list_of_engine_size = set(list_of_engine_size)

In [ ]:
# Print the engine sizes to check if they are correct
print(unique_list_of_engine_size)

In [ ]:
# Export the result to .csv file
# df_year_body_engine_size_removed_char.to_csv('C:\Language_Projects\Language_Projects\Python\Flagship_1\pdf_data_extract\data\chrysler_pdf_extract_to_csv.csv')